Import all the needed library

In [2]:
from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.regularizers import l2

from PIL import Image
import scipy.misc

In [3]:
folders = listdir("../")
folders

['.ipynb_checkpoints',
 'scr',
 'NormalMerged.txt',
 'CancerMerged.txt',
 'cancer_image',
 'normal_image']

In [4]:
cancer_filename = "../CancerMerged.txt"
normal_filename = "../NormalMerged.txt"

In [5]:
cancer_data = pd.read_table(cancer_filename, header=None)
cancer_data_t = cancer_data.T
cancer_data_tmp = cancer_data_t
cancer_data_tmp.columns = cancer_data_tmp.iloc[0]
cData = cancer_data_tmp.drop(0)
cData.shape

(3104, 60488)

In [6]:
normal_data = pd.read_table(normal_filename, header=None)
normal_data_t = normal_data.T
normal_data_tmp = normal_data_t
normal_data_tmp.columns = normal_data_tmp.iloc[0]
nData = normal_data_tmp.drop(0)
nData.shape

(324, 60488)

In [7]:
allData = pd.concat([cData, nData], ignore_index=True)

In [8]:
#d = np.array(allData.values, dtype='float')
dat = allData.values
dat = dat.astype('float32')
dat[np.isnan(dat)] = 0

---

In [ ]:
b = [0.0] * 12
for i in range(3104):
    tmp = cData.iloc[i].tolist()
    tmp = np.array(tmp + b)
    tmp = tmp + 1
    #tmp = np.log10(tmp)
    tmp_2d = tmp.reshape(250, 242)
    name = "../cancer_image/c"+("%04d" % (i,)) +".jpg"
    scipy.misc.imsave(name, img)
    
a = cData.iloc[0].tolist()

In [ ]:
b = [0.0] * 12
c = np.array(a + b)
c = c + 1
img = np.log10(c)
#img = c
plt.hist(img)
plt.show()
img = img.reshape(250, 242)

In [ ]:
type(img)

In [ ]:
plt.imshow(img)
#plt.colorbar()
plt.show()


scipy.misc.imsave('outfile.jpg', img)

In [ ]:
"c"+("%04d" % (1,)) +".jpg"

Change data to 250*242=60500

In [ ]:
b = [0.0] * 12

for i in range(324):
    tmp = None
    tmp = nData.iloc[i].tolist()
    # print(tmp)
    tmp = np.array(tmp + b)
    tmp = tmp + 1
    tmp = np.log10(tmp)
    tmp_2d = tmp.reshape(250, 242)
    name = "../normal_image/n"+("%04d" % (i,)) +".jpg"
    scipy.misc.imsave(name, img)

In [ ]:
def img_gen(input_d):
    b = [0.0] * 12
    tmp = np.array(input_d + b)
    tmp = tmp + 1
    tmp = np.log10(tmp)
    tmp_2d = tmp.reshape(250, 242)

## To show Nathan

In [ ]:
b = [0.0] * 12
import csv
for i in range(10):
    tmp = cData.iloc[i].tolist()
    tmp = np.array(tmp + b).reshape(250, 242)
    fname = "../scr/Cancer/c"+("%04d" % (i,)) + ".csv"
    np.savetxt(fname, tmp, delimiter=',')
    
    tmp_2d = tmp.reshape(250, 242)
    name = "../scr/Cancer/c"+("%04d" % (i,)) +"_original.jpg"
    scipy.misc.imsave(name, img)
    
    tmp = tmp + 1
    tmp = np.log10(tmp)
    tmp_2d = tmp.reshape(250, 242)
    name = "../scr/Cancer/c"+("%04d" % (i,)) +"_log.jpg"
    scipy.misc.imsave(name, img)

In [ ]:
for i in range(10):
    tmp = nData.iloc[i].tolist()
    tmp = np.array(tmp + b).reshape(250, 242)
    fname = "../scr/Normal/n"+("%04d" % (i,)) + ".csv"
    np.savetxt(fname, tmp, delimiter=',')
    
    tmp_2d = tmp.reshape(250, 242)
    name = "../scr/Normal/n"+("%04d" % (i,)) +"_original.jpg"
    scipy.misc.imsave(name, img)
    
    tmp = tmp + 1
    tmp = np.log10(tmp)
    tmp_2d = tmp.reshape(250, 242)
    name = "../scr/Normal/n"+("%04d" % (i,)) +"_log.jpg"
    scipy.misc.imsave(name, img)

In [ ]:
tmp1 = nData.iloc[1].tolist()
tmp1 = np.array(tmp1 + b)
print(tmp1.tolist())

In [ ]:
tmp2 = nData.iloc[2].tolist()
tmp2 = np.array(tmp2 + b)
print(tmp2.tolist())

# May 3
Nathan suggests: 
1. Solve for a PCA basis and project on to that.
2. Project onto a random basis (instead of PCA make each entry in the projection matrix an independent sample from a unit Gaussian).
3. Start with 1) or 2) and then optimize the projection for the task (but you will have to be careful here that you don't overfit since you will have so many free parameters).  Heavy regularization might be necessary.

In [12]:
# 1. Solve for a PCA basis
allData = pd.concat([cData, nData], ignore_index=True)

NameError: name 'nData' is not defined

In [ ]:
allData.shape


In [ ]:
d[:,0:3428].shape

In [ ]:
results = np.linalg.svd(d)

In [ ]:
results

# Audoendoder

In [9]:
from keras.layers import Input, Dense
from keras.models import Model

In [25]:
encoding_dim = 250 # 32 floats -> compression of factor 250, assuming the input is 60500 floats

# this is our input placeholder
input_img = Input(shape=(60488,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(60488, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

encoder model

In [26]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

decoder model

In [27]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

let's train our autoencoder to reconstruct data.

First, we'll configure our model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer:



In [40]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy', 'mae'])

Find which one is nan

In [12]:
from sklearn.cross_validation import train_test_split
x_train, x_test = train_test_split(dat,
                                     test_size=0.2,
                                     random_state=0)

/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
results = autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 2742 samples, validate on 686 samples
Epoch 1/50
2742/2742 [==============================] - 15s - loss: -667317.1864 - acc: 0.3813 - mean_absolute_error: 42195.6903 - val_loss: -657023.3808 - val_acc: 0.3958 - val_mean_absolute_error: 41639.7109

KeyboardInterrupt: 

After 50 epochs, the autoencoder seems to reach a stable train/test loss value of about 0.11. We can try to visualize the reconstructed inputs and the encoded representations. We will use Matplotlib.

In [ ]:
# encode and decode some file
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# May 8

## Adding a sparsity constraint on the encoded representations

In the previous example, the representations were only constrained by the size of the hidden layer (32). In such a situation, what typically happens is that the hidden layer is learning an approximation of PCA (principal component analysis). But another way to constrain the representations to be compact is to add a sparsity contraint on the activity of the hidden representations, so fewer units would "fire" at a given time. In Keras, this can be done by adding an activity_regularizer to our Dense layer:

In [10]:
from keras import regularizers


encoding_dim = 250 # 32 floats -> compression of factor 250, assuming the input is 60500 floats

# this is our input placeholder
input_img = Input(shape=(60488,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(60488, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)


In [44]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy', 'mae'])
results2 = autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 2742 samples, validate on 686 samples
Epoch 1/50
2742/2742 [==============================] - 16s - loss: -281017.6299 - acc: 0.3082 - mean_absolute_error: 42195.8029 - val_loss: -452354.4852 - val_acc: 0.3341 - val_mean_absolute_error: 41639.7822

KeyboardInterrupt: 

### Adding more layers -- Deep autoencoder

In [13]:
input_img = Input(shape=(60488,))
encoded = Dense(1000, activation='relu')(input_img)
encoded = Dense(500, activation='relu')(encoded)
encoded = Dense(250, activation='relu')(encoded)

decoded = Dense(250, activation='relu')(encoded)
decoded = Dense(500, activation='relu')(encoded)
decoded = Dense(1000, activation='relu')(decoded)
decoded = Dense(60488, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy', 'mae'])

result3 = autoencoder.fit(x_train, x_train,
                epochs=1000,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 2742 samples, validate on 686 samples
Epoch 1/1000
2742/2742 [==============================] - 36s - loss: -583345.7957 - acc: 0.3704 - mean_absolute_error: 42195.7324 - val_loss: -649403.0275 - val_acc: 0.3953 - val_mean_absolute_error: 41639.7177

# Note:

## LSTM autoencoder on sequences of text (titles of web articles)

In [ ]:
input_sentence = Input(shape=(max_title_length, number_of_chars), dtype='int32')
tofloat = Lambda(function=lambda x: tf.to_float(x))(input_sentence)
encoder = LSTM(latent_dim, activation='tanh')(tofloat)

decoder = RepeatVector(max_title_len)(encoder)
decoder = LSTM(number_of_chars, return_sequences=True, activation='tanh')(decoder)
autoencoder = Model(input=input_sentence, output=decoder)

autoencoder.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Deep autoencoder

We do not have to limit ourselves to a single layer as encoder or decoder, we could instead use a stack of layers, such as:

In [ ]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## Convolutional autoencoder

Since our inputs are images, it makes sense to use convolutional neural networks (convnets) as encoders and decoders. In practical settings, autoencoders applied to images are always convolutional autoencoders --they simply perform much better.

Let's implement one. The encoder will consist in a stack of Conv2D and MaxPooling2D layers (max pooling being used for spatial down-sampling), while the decoder will consist in a stack of Conv2D and UpSampling2D layers.


In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

To train it, we will use the original MNIST digits with shape (samples, 3, 28, 28), and we will just normalize pixel values between 0 and 1.

In [ ]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

Let's train this model for 50 epochs. For the sake of demonstrating how to visualize the results of a model during training, we will be using the TensorFlow backend and the TensorBoard callback.

First, let's open up a terminal and start a TensorBoard server that will read logs stored at /tmp/autoencoder.

In [ ]:
tensorboard --logdir=/tmp/autoencoder

Then let's train our model. In the callbacks list we pass an instance of the TensorBoard callback. After every epoch, this callback will write logs to /tmp/autoencoder, which can be read by our TensorBoard server.

In [ ]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

This allows us to monitor training in the TensorBoard web interface (by navighating to http://0.0.0.0:6006):

The model converges to a loss of 0.094, significantly better than our previous models (this is in large part due to the higher entropic capacity of the encoded representation, 128 dimensions vs. 32 previously). Let's take a look at the reconstructed digits:



In [ ]:
decoded_imgs = autoencoder.predict(x_test)

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

We can also have a look at the 128-dimensional encoded representations. These representations are 8x4x4, so we reshape them to 4x32 in order to be able to display them as grayscale images.

In [ ]:
n = 10
plt.figure(figsize=(20, 8))
for i in range(n):
    ax = plt.subplot(1, n, i)
    plt.imshow(encoded_imgs[i].reshape(4, 4 * 8).T)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Train model with Y

In [ ]:
x = Input(shape=(input_dim,))
encoder = Dense(output_dim)(x)
decoder = Dense(input_dim)(encoder)
 
autoencoder = Model(input=x, output=decoder)
supervised = Model(input=x, output=encoder)
 
autoencoder.compile(...)
supervised.compile(...)


## MINST Data

In [ ]:
from keras.datasets import mnist
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)